# SportsStats Surfece Level Data Exploration

**Table of Contents**

1. Introduction
2. Load Cleaned Data
3. Data Exploration
4. Summary of Statistics
5. Entity Relationship Diagram

---

## I. Introduction

The client I will be working with is SportsStats, a sports analysis firm partnering with local news and elite personal trainers to provide insights to help their partners. They recognize patterns/trends highlighting certain groups, events, countries, etc. for the purpose of developing a news story or discovering key health insights. As someone with a personal history with several sports such as basketball, martial arts, fencing, bicycling, and more, being able to perform analysis to gain insights on sports is something that personally engages me. Additionally, I reside within a culture that is heavily influenced by health and well-being as an ideal to pursue, so I find that this analysis that will be able to provide key health insights will provide value not only to the client I will be working with in this project, but also to the general public as well.

The Data Quality Assessment notebook provides the methodology and results for discovering issues with the raw data provided by SportsStats, and the Data Cleaning Procedure notebook provides the methodology for resolving most of the issues as well as access to cleaned versions of the datasets. The cleaned version of the dataset is precisely what I will be using for Surface Level Data Exploration to provide context for the dataset we will be working with in future analysis.

---
    
## II. Load Cleaned Data

First, you may desire to understand, and potentially adjust your working directory. Understanding from which environment you are working from is critical for your project, especially if you intend to replicate the steps provided. The code that will be provided after this step assumes you are working with a relative path to retrieve, load, and use CSV and DB files stored under the same folder as where this notebook would be saved. As such, make sure that your current working directory matches accordingly.

### Working Directory (Optional)

In [1]:
# Import the os library
import os

In [2]:
# Then use the following code to find out where your current working directory is
print(os.getcwd())

C:\Users\randy\OneDrive\Documents\Data Analytics\SportsStats Analysis


In [3]:
# If the cwd location for your project is not where you would prefer it to be, 
# use something akin to the following to change it:
os.chdir(r"C:\Users\randy\OneDrive\Documents\Data Analytics\SportsStats Analysis")
print(os.getcwd())

C:\Users\randy\OneDrive\Documents\Data Analytics\SportsStats Analysis


### Importing Libraries

Before working on importing the data, import the libraries to use for the project. 
- Pandas will be used to import the data from the CSV files provided. 
- SQLite3 will be used to import Jupyter compatible extensions that will enable you to code and query in SQLite.
- re will be used to create a function into SQLite that will enable the usage of Regular Expressions (or Regex).

In [4]:
# Importing the libraries
import pandas as pd
import sqlite3
import re

### Importing the data into Jupyter Notebook

Make sure the CSV files `athlete_events.csv` and `noc_regions.csv` are stored within your current working directory first before proceeding.

In [5]:
# Reading the CSV Files as Dataframes
ath_events = pd.read_csv("athlete_events_cleaned.csv")
noc_regions = pd.read_csv("noc_regions_cleaned.csv")

In [6]:
# Testing the success of reading:
print(ath_events.head(3))
print(noc_regions.head(3))

   ID                 Name Sex   Age  Height  Weight     Team  NOC  \
0   1            A Dijiang   M  24.0   180.0    80.0    China  CHN   
1   2             A Lamusi   M  23.0   170.0    60.0    China  CHN   
2   3  Gunnar Nielsen Aaby   M  24.0     NaN     NaN  Denmark  DEN   

         Games  Year  Season       City       Sport  \
0  1992 Summer  1992  Summer  Barcelona  Basketball   
1  2012 Summer  2012  Summer     London        Judo   
2  1920 Summer  1920  Summer  Antwerpen    Football   

                          Event     Medal  
0   Basketball Men's Basketball  No Medal  
1  Judo Men's Extra-Lightweight  No Medal  
2       Football Men's Football  No Medal  
   NOC       region                 notes
0  AFG  Afghanistan                   NaN
1  AHO      Curacao  Netherlands Antilles
2  ALB      Albania                   NaN


### Importing the data from Jupyter into SQL Database

We are going to create a connection between the data we have read thus far and a SQLite database. This code can either create a database file if one does not exist under the name you chose or connect the database file that exists with the same name.

In [7]:
## Loading of Data into SQLite
conn = sqlite3.connect('olympics.db')
ath_events.to_sql('ath_events', conn, if_exists='replace', index=False)
noc_regions.to_sql('noc_regions', conn, if_exists='replace', index=False)

# The result of this would be a number. This number corresponds to the number of rows
# existing within the most recent dataset you connected to with your sqlite database.
# We previously set up a database for sqlite called "olympics.db". From there, we
# connected that database to the data we have listed over here.

230

### Loading SQL Extensions and Regex to run SQL Queries

We will be using SQLite code to handle the data from here on out, but first, we need to install jupysql into the notebook itself if not installed yet. From there, we load the SQL extension into your notebook, connect to the olympics.db, configure so that the display limit is turned off, and establish the REGEXP function that will allow for the use of Regex during assessment.

In [8]:
# Install jupysql (Only run code once. If already installed, no need to install again)
#!pip install jupysql

In [9]:
# Load the SQL extension
%load_ext sql

In [10]:
# Connect to olympics.db database
%sql sqlite:///olympics.db

Connecting to 'sqlite:///olympics.db'

In [11]:
# If not yet configured so that the display limit no longer is active, this code turns it off:
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [12]:
# Define the REGEXP function
def regexp(pattern, value):
    if value is None:
        return False
    return re.search(pattern, value) is not None

# Register the function with SQLite
conn.create_function("REGEXP", 2, regexp)

*A note before proceeding with the surface level exploration, most of the code below will provide outputs for the first three rows simply for presentation purposes. To properly understand the data and its quality, you may need to alter the code to provide more rows.*

---

## III. Data Exploration

We will separate this portion into six sections:

1. Dataset Overview
2. Temporal Distribution                                                                                                                                3. Athlete Demographics                                                                                    
4. Geographic Representation                                                                                                                            5. Sports and Events                                                                                        
6. Medal Distribution

### i. Dataset Overview                                                                                                                                 
                                                                                                                                                        
To gain an overview of the datasets, we will be observing the table schemas, an overlook of the primary dataset `ath_events` and an overlook of the reference datase `noc_regions`.                                                                                                                                                                                                                    
**Table Schemas:**                                                                                                                                      

In [13]:
%%sql
-- Info on all the column names and data types of ath_events
PRAGMA table_info(ath_events)

Running query in 'sqlite:///olympics.db'

cid,name,type,notnull,dflt_value,pk
0,ID,INTEGER,0,None,0
1,Name,TEXT,0,None,0
2,Sex,TEXT,0,None,0
3,Age,REAL,0,None,0
4,Height,REAL,0,None,0
5,Weight,REAL,0,None,0
6,Team,TEXT,0,None,0
7,NOC,TEXT,0,None,0
8,Games,TEXT,0,None,0
9,Year,INTEGER,0,None,0


In [14]:
%%sql
-- Info on all the column names and data types of noc_regions
PRAGMA table_info(noc_regions)

Running query in 'sqlite:///olympics.db'

cid,name,type,notnull,dflt_value,pk
0,NOC,TEXT,0,None,0
1,region,TEXT,0,None,0
2,notes,TEXT,0,None,0


The queries above shows the information about the tables that you will need to know. Note that the only INTEGER data types are for ID and Year, and the only REAL data types (which allows for decimal calculation) are for Age, Height, and Weight, but you will see that the values of those columns are expressed with whole numbers. The rest of the columns contain TEXT data.
                                                                                                                                                        **Primary Dataset**                                                               

In [15]:
%%sql
-- Number of records of ath_events
SELECT COUNT(*) FROM ath_events

Running query in 'sqlite:///olympics.db'

COUNT(*)
269661


In [16]:
%%sql
-- Number of athletes (by ID)
SELECT COUNT(DISTINCT ID) FROM ath_events

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT ID)
135571


In [17]:
%%sql
-- Time Span of Data
SELECT MIN(Year), MAX(Year) FROM ath_events

Running query in 'sqlite:///olympics.db'

MIN(Year),MAX(Year)
1896,2016


In [18]:
%%sql
-- Number of games
SELECT COUNT(DISTINCT Games) FROM ath_events

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Games)
51


In [19]:
%%sql
-- Number of summer games
SELECT COUNT(DISTINCT Games) FROM ath_events WHERE Season = "Summer"

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Games)
29


In [20]:
%%sql
-- Number of summer games
SELECT COUNT(DISTINCT Games) FROM ath_events WHERE Season = "Winter"

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Games)
22


The queries above shows some information you will need to know about the size of the dataset and what it encompasses.

**Reference Dataset**

In [21]:
%%sql
-- Number of records of noc_regions
SELECT COUNT(*) FROM noc_regions

Running query in 'sqlite:///olympics.db'

COUNT(*)
230


In [22]:
%%sql
-- Number of NOCs
SELECT COUNT(DISTINCT NOC) FROM noc_regions

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT NOC)
230


In [23]:
%%sql
-- Number of Regions
SELECT COUNT (DISTINCT Region) FROM noc_regions

Running query in 'sqlite:///olympics.db'

COUNT (DISTINCT Region)
209


The queries above shows some information pertaining to National Olympic Committees and their associations with Regions across the world.

To summarize the results of the queries above:

**Table Schemas**
- INTEGER: ID, Year
- REAL: Age, Height, Weight
- TEXT (ath_events): Name, Sex, Team, NOC, Games, Season, City, Sport, Event, Medal
- TEXT (noc_regions): NOC, Region, notes

**Primary Dataset (ath_events)**

- Total records: 269,661
- Unique Athletes: 135,571
- Time Span: 1896-2016 (120 Years)
- Temporal Coverage: 51 Olympic Games (29 Summer, 22 Winter)

**Reference Dataset (noc_regions)**

- Total records: 230
- National Olympic Committees (NOCs): 230
- Regions Represented: 209

### ii. Temporal Distribution

We will look at games by era, noting how many games are there within each era, what years the eras consist of, and how many distinct athletes participated in each era. Additionally, we will make note of any key milestones that contextualize how the games functioned across history.

In [24]:
%%sql
SELECT Games, COUNT(DISTINCT ID)
FROM ath_events
GROUP BY Games
ORDER BY Games
LIMIT 3

Running query in 'sqlite:///olympics.db'

Games,COUNT(DISTINCT ID)
1896 Summer,176
1900 Summer,1224
1904 Summer,650


In [25]:
%%sql
-- Early Era
SELECT COUNT(DISTINCT Games), COUNT(DISTINCT ID)
FROM ath_events
WHERE Year >= 1896 AND Year <= 1912

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Games),COUNT(DISTINCT ID)
6,6637


In [26]:
%%sql
-- Interwar Period
SELECT COUNT(DISTINCT Games), COUNT(DISTINCT ID)
FROM ath_events
WHERE Year >= 1920 AND Year <= 1936

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Games),COUNT(DISTINCT ID)
9,14897


In [27]:
%%sql
-- Post-WWII
SELECT COUNT(DISTINCT Games), COUNT(DISTINCT ID)
FROM ath_events
WHERE Year >= 1948 AND Year <= 1992

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Games),COUNT(DISTINCT ID)
24,65166


In [28]:
%%sql
-- Modern Era
SELECT COUNT(DISTINCT Games), COUNT(DISTINCT ID)
FROM ath_events
WHERE Year >= 1994 AND Year <= 2016

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Games),COUNT(DISTINCT ID)
12,54201


Summary of what needs to be understood about the temporal distribution of the Olympic Games:

**Games by Era:**
- Early Era (1896-1912): 6 games, 6,637 athletes
- Interwar Period (1920-1936): 9 games, 14,897 athletes
- Post-WWII Non-Staggered (1948-1992): 24 games, 65,166 athletes
- Modern Era (1994-2016): 12 games, 54,201 athletes

**Key Milestone:**
- Only Intercalated Olympics during early 1900s: 1906
- First Winter Games: 1924
- Distribution: Summer and Winter Games mostly occurr every 4 years with exception of years during WWI and WWII
- Games Staggered (2-year cycle between Summer and Winter games): 1994 onwards

*Note: If you derive the number of athletes for each era by manually adding the number of distinct IDs for each year with each sum corresponding to each era, you will find yourself with more athletes than what should be calculated. This is the result of the same athletes competing across multiple games. That being said, calculating the information through SQL with the above queries will provide you with the actual distinct number of athletes that participated in each era.*

### iii. Athlete Demographics

We will look at gender representation, age distribution, and physical attributes height and weight where recorded. We will also take this opportunity to recognize outliers defined by the IQR calculation. Considering the Interquartile Range (IQR) to be the difference between the first quartile and the third quartile (25th percentile and 75th percentile respectively), a numerical outlier in a dataset is considered an outlier when it is either 1.5 IQRs below the first quartile or 1.5 IQRs above the third quartile.

**Gender Representation**

In [29]:
%%sql
-- Query for Gender Representation
-- Note that when calculating percentages, 
-- you must multiply the numerator by a number which considers decimals before dividing, 
-- otherwise it truncates to 0.
SELECT Sex, COUNT(Sex) AS "Records by Sex", (COUNT(*)*100.0/(SELECT COUNT(*) FROM ath_events)) AS "Percentage by Sex"
FROM ath_events
GROUP BY Sex

Running query in 'sqlite:///olympics.db'

Sex,Records by Sex,Percentage by Sex
F,74375,27.580925680762142
M,195286,72.41907431923786


The above query shows the percentage of times a contestant participates in an event from the Olympics. Note that this is different from the distinct number of athletes who participated in the Olympics partitioned by sex. That is something we can explore in future analysis.

**Age Distribution**

In [30]:
%%sql
-- Query for Age distribution through a manual calculation of the interquartile range:
-- First, order the sequence of age from bottom to top, removing null values in the process
-- Second, look for the min, max, and percentiles 25, 50, and 75, which describe the quartiles Q1, Q2 (Median), and Q3
WITH ordered AS (
    SELECT
        Age,
        ROW_NUMBER() OVER (ORDER BY Age) AS rn,
        COUNT(*) OVER () AS total
    FROM ath_events
    WHERE Age IS NOT NULL
)
SELECT
  (SELECT MIN(Age) FROM ath_events) AS Min,
  (SELECT Age FROM ordered WHERE rn = CAST(total * 0.25 AS INT)) AS Q1,
  (SELECT Age FROM ordered WHERE rn = CAST(total * 0.50 AS INT)) AS Median,
  (SELECT Age FROM ordered WHERE rn = CAST(total * 0.75 AS INT)) AS Q3,
  (SELECT MAX(Age) FROM ath_events) AS Max

Running query in 'sqlite:///olympics.db'

Min,Q1,Median,Q3,Max
10.0,21.0,24.0,28.0,97.0


The above query shows the distribution of Age in terms of quartiles. From this information, we can determine that the IQR is 7 years (Q3-Q1 = 28-21), and outliers would be considered outliers if they are 1.5*IQR (or 10.5 years) below Q1 or above Q3. That means any age below 10.5 or above 38.5 is considered an outlier.

In [31]:
%%sql
-- Outliers below
SELECT COUNT(*)
FROM ath_events
WHERE Age <= 10.5;

Running query in 'sqlite:///olympics.db'

COUNT(*)
1


In [32]:
%%sql
-- Outliers above
SELECT COUNT(*)
FROM ath_events
WHERE Age >= 38.5

Running query in 'sqlite:///olympics.db'

COUNT(*)
9405


There is only one instance of a participant being an outlier for being younger than most participants, but there are over 9,000 instances of participants being an outlier for being older than most participants. It's safe to say that the age distribution is skewed right rather than center balanced given this information. We will also want to see how much of our records contain Age data for analysis.

In [33]:
%%sql
-- Query for % of rows with Age
SELECT COUNT(Age) * 100.0 / COUNT(*) AS "Percentage of Records with Age data"
FROM ath_events;

Running query in 'sqlite:///olympics.db'

Percentage of Records with Age data
96.56160883479627


We have about 96.56% of the column full of age data.

**Physical Attributes**

In [34]:
%%sql
-- Query for Height distribution through a manual calculation of the interquartile range:
-- First, order the sequence of Height from bottom to top, removing null values in the process
-- Second, look for the min, max, and percentiles describing the quartiles Q1, Q2 (Median), and Q3
WITH ordered AS (
    SELECT
        Height,
        ROW_NUMBER() OVER (ORDER BY Height) AS rn,
        COUNT(*) OVER () AS total
    FROM ath_events
    WHERE Height IS NOT NULL
)
SELECT
  (SELECT MIN(Height) FROM ath_events) AS Min,
  (SELECT Height FROM ordered WHERE rn = CAST(total * 0.25 AS INT)) AS Q1,
  (SELECT Height FROM ordered WHERE rn = CAST(total * 0.50 AS INT)) AS Median,
  (SELECT Height FROM ordered WHERE rn = CAST(total * 0.75 AS INT)) AS Q3,
  (SELECT MAX(Height) FROM ath_events) AS Max

Running query in 'sqlite:///olympics.db'

Min,Q1,Median,Q3,Max
127.0,168.0,175.0,183.0,226.0


The above query shows the distribution of Height in terms of quartiles. From this information, we can determine that the IQR is 15cm (Q3-Q1 = 183-168), and outliers would be considered outliers if they are 1.5*IQR (or 22.5 cm) below Q1 or above Q3. That means any height below 145.5 or above 205.5 is considered an outlier.

In [35]:
%%sql
-- Outliers below
SELECT COUNT(*)
FROM ath_events
WHERE Height <= 145.5;

Running query in 'sqlite:///olympics.db'

COUNT(*)
701


In [36]:
%%sql
-- Outliers above
SELECT COUNT(*)
FROM ath_events
WHERE Height >= 205.5;

Running query in 'sqlite:///olympics.db'

COUNT(*)
637


We see a total of 1,338 outliers for Height, with 701 outliers below most participants' height and 637 outliers above most participants' height. While it could be interpreted that there is a slight skew to the left, it's far more balanced than the skew from Age.

In [37]:
%%sql
-- Query for Weight distribution through a manual calculation of the interquartile range:
-- First, order the sequence of Weight from bottom to top, removing null values in the process
-- Second, look for the min, max, and percentiles 25, 50, and 75, which describe Q1, Q2 (Median), and Q3
WITH ordered AS (
    SELECT
        Weight,
        ROW_NUMBER() OVER (ORDER BY Weight) AS rn,
        COUNT(*) OVER () AS total
    FROM ath_events
    WHERE Weight IS NOT NULL
)
SELECT
  (SELECT MIN(Weight) FROM ath_events) AS Min,
  (SELECT Weight FROM ordered WHERE rn = CAST(total * 0.25 AS INT)) AS Q1,
  (SELECT Weight FROM ordered WHERE rn = CAST(total * 0.50 AS INT)) AS Median,
  (SELECT Weight FROM ordered WHERE rn = CAST(total * 0.75 AS INT)) AS Q3,
  (SELECT MAX(Weight) FROM ath_events) AS Max

Running query in 'sqlite:///olympics.db'

Min,Q1,Median,Q3,Max
25.0,60.0,70.0,79.0,214.0


The above query shows the distribution of Weight in terms of quartiles. From this information, we can determine that the IQR is 19kg (Q3-Q1 = 79-60), and outliers would be considered outliers if they are 1.5*IQR (or 28.5 kg) below Q1 or above Q3. That means any weight below 31.5 or above 107.5 is onsidered an outlier.

In [38]:
%%sql
-- Outliers below
SELECT COUNT(*)
FROM ath_events
WHERE Weight <= 31.5;

Running query in 'sqlite:///olympics.db'

COUNT(*)
85


In [39]:
%%sql
-- Outliers above
SELECT COUNT(*)
FROM ath_events
WHERE Weight >= 107.5;

Running query in 'sqlite:///olympics.db'

COUNT(*)
3154


We see a total of 3,239 outliers for Weight, with 85 outliers below most participants' weight and 3154 outliers above most participants' weight. From this, we are able to determine that the distribution of weight across the data is skewed to the right. From here, we want to see how much of our records actually contain data in our datasets for analysis.

In [40]:
%%sql
-- Query for % of rows with Height
SELECT COUNT(Height) * 100.0 / COUNT(*) AS "Percentage of Records with Height data"
FROM ath_events;

Running query in 'sqlite:///olympics.db'

Percentage of Records with Height data
78.22080315655582


In [41]:
%%sql
-- Query for % of rows with Weight
SELECT COUNT(Weight) * 100.0 / COUNT(*) AS "Percentage of Records with Weight data"
FROM ath_events;

Running query in 'sqlite:///olympics.db'

Percentage of Records with Weight data
77.21472515491674


We have about 78.22% of the Height column full of height data and 77.21% of the Weight column full of weight data.

**Summary of Athlete Demographics:**

**Gender Representation**

- Male: 195,286 records (72.42%)
- Female: 74,375 records (27.58%)
    
**Age Distribution**

- Youngest Competitor: 10 years old
- Oldest Competitor: 97 years old
- Median Age: 24 years
- IQR of Age: 7 years (with 1st quartile at 21 years and 3rd quartile at 28 years)
- Outliers: 9,406 (1 to the left, 9,405 to the right)
- Distribution: Skewed Right
- Data completion: 96.56%
                          
**Physical Attributes**

Height
- Shortest Competitor: 127cm
- Tallest Competitor: 226cm
- Median Height: 175cm
- IQR of Height: 15cm (with 1st quartile at 168cm and 3rd quartile at 183cm)
- Outliers: 1,338 (701 to the left, 637 to the right)
- Distribution: Balanced (Slightly Skewed Left) 
- Data completion: 78.22%

Weight
- Lightest Competitor: 25kg
- Heaviest Competitor: 214kg
- Median Weight: 70kg
- IQR of Weight: 19kg (with 1st quartile at 60kg and 3rd quartile at 79kg)
- Outliers: 3,239 (85 to the left, 3,154 to the right)
- Distribution: Skewed Right
- Data completion: 77.21%

### iv. Geographic Representation

When it comes to geographic representation, the columns of interest are Team, NOC, and Region for the organizations and groups participating in the Olympics and City for hosting the Olympics. We will be looking at the counts for distinct values for these columns as well as the counts for the NOCs that have the most teams associated with them.

In [42]:
%%sql
-- Counts for distinct values
SELECT COUNT(DISTINCT Team), COUNT(DISTINCT NOC), COUNT(DISTINCT City)
FROM ath_events

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Team),COUNT(DISTINCT NOC),COUNT(DISTINCT City)
1184,230,42


In [43]:
%%sql
-- Obtained by joining merging earlier team_noc query from data cleaning with regions from noc_regions table
WITH team_noc (team, noc) AS (
    SELECT DISTINCT Team, NOC
    FROM ath_events
)
SELECT tn.noc AS "NOC", nr.region ,COUNT(tn.team) AS "Number of Teams Associated"
FROM team_noc tn
LEFT JOIN noc_regions nr
ON tn.noc = nr.NOC
GROUP BY tn.noc
ORDER BY COUNT(team) DESC
LIMIT 10;

Running query in 'sqlite:///olympics.db'

NOC,region,Number of Teams Associated
FRA,France,162
USA,USA,97
GBR,UK,94
SWE,Sweden,52
NOR,Norway,46
GER,Germany,43
ITA,Italy,36
CAN,Canada,32
GRE,Greece,29
DEN,Denmark,29


**Summary of Geographic Representation**

- Number of Distinct Teams: 1184
- Number of Distinct NOCs: 230
- Number of Distinct Regions: 209
- Number of Cities that hosted the Olympics: 42

**<center>Top 10 Regions by Team Associations:</center>**

| Region  | Number of Teams Associated |
|---------|----------------------------| 
| France  | 162                        |
| USA     | 97                         |
| UK      | 94                         |
| Sweden  | 52                         |
| Norway  | 46                         |
| Germany | 43                         |
| Italy   | 36                         |
| Canada  | 32                         |
| Greece  | 29                         |
| Denmark | 29                         |

### v. Sports and Events

We will account for the number of distinct sports, events, consider discontinued sports, and consider which ones are for the summer and which for the winter. Additionally, we will observe the sports holding the most events.    

In [44]:
%%sql
SELECT COUNT(DISTINCT Sport), COUNT(DISTINCT Event)
FROM ath_events

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Sport),COUNT(DISTINCT Event)
66,765


In [45]:
%%sql
SELECT COUNT(DISTINCT Sport)
FROM ath_events
WHERE Season = "Summer"

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Sport)
52


In [46]:
%%sql
SELECT COUNT(DISTINCT Sport)
FROM ath_events
WHERE Season = "Winter"

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Sport)
17


In [47]:
%%sql
WITH current_sports AS (
    SELECT DISTINCT Sport
    FROM ath_events
    WHERE Year >= 1994
)
SELECT COUNT(DISTINCT Sport)
FROM ath_events
WHERE Sport NOT IN current_sports;

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Sport)
15


In [48]:
%%sql
WITH current_sports AS (
    SELECT DISTINCT Sport
    FROM ath_events
    WHERE Year >= 1994
)
SELECT COUNT(DISTINCT Sport)
FROM ath_events
WHERE Season = "Summer" AND Sport IN current_sports

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Sport)
38


In [49]:
%%sql
WITH current_sports AS (
    SELECT DISTINCT Sport
    FROM ath_events
    WHERE Year >= 1994
)
SELECT COUNT(DISTINCT Sport)
FROM ath_events
WHERE Season = "Winter" AND Sport IN current_sports

Running query in 'sqlite:///olympics.db'

COUNT(DISTINCT Sport)
15


In [50]:
%%sql
SELECT Sport, COUNT(DISTINCT Event) AS "Number of Distinct Events in Sport"
FROM ath_events
GROUP BY Sport
ORDER BY COUNT(DISTINCT Event) DESC
LIMIT 5

Running query in 'sqlite:///olympics.db'

Sport,Number of Distinct Events in Sport
Shooting,83
Athletics,83
Swimming,55
Cycling,44
Sailing,38


**Sport Coverage:**
- Total Sports: 66
- Total Events: 765
- Average Events per Sport: ~11.6

**Sport Categories:**
- Summer Sports: 52 (including discontinued sports, 38 if excluding)
- Winter Sports: 17 (including discontinued sports, 15 if excluding)
- Discontinued Sports: 15 (e.g., Art Competitions, Tug-Of-War)

**<center>Most Event-Dense Sports:</center>**
    
|Sport|Number of Distinct Events in Sport|
|-|-|
|Shooting|83|
|Athletics|83|
|Swimming|55|
|Cycling|44|
|Sailing|38|

### vi. Medal Distribution

We will observe the number of Bronze, Silver, Gold, and No Medal records, noting any potential discrepancies

In [51]:
%%sql
SELECT Medal, COUNT(Medal)
FROM ath_events
GROUP BY Medal

Running query in 'sqlite:///olympics.db'

Medal,COUNT(Medal)
Bronze,13287
Gold,13368
No Medal,229906
Silver,13100


**Overall Medal Awards:**
- Gold Medals: 13,368
- Silver Medals: 13,100
- Bronze Medals: 13,287
- No Medal: 229,906 records

**Note on Bronze/Silver Discrepancy:**
Bronze medals slightly exceed silver medals, likely due to:
- Sports with third-place playoffs (e.g., team sports)
- Events awarding multiple bronze medals for tied competitors
- Historical variations in medal award protocols

## IV. Summary of Statistics

**Primary Dataset (ath_events)**

- Total records: 269,661
- Unique Athletes: 135,571
- Time Span: 1896-2016 (120 Years)
- Temporal Coverage: 51 Olympic Games (29 Summer, 22 Winter)

**Reference Dataset (noc_regions)**

- Total records: 230
- National Olympic Committees (NOCs): 230
- Regions Represented: 209

**Games by Era:**
- Early Era (1896-1912): 6 games, 6,637 athletes
- Interwar Period (1920-1936): 9 games, 14,897 athletes
- Post-WWII Non-Staggered (1948-1992): 24 games, 65,166 athletes
- Modern Era (1994-2016): 12 games, 54,201 athletes

**<center>Athlete Demographics</center>**

| Column | Min | Q1 | Median | Q3 | Max | Outliers | Distribution | Data Completion |
| ------ | --- | -- | ------ | -- | --- | -------- | ------------ | --------------- |
| Age    | 10y | 21y | 24y | 28y | 97y | 9,406 recs | Skewed Right | 96.56% |
| Height | 127cm | 168cm | 175cm | 183cm | 226cm | 1,338 recs | Balnced (Slightly Skewed Left) | 78.22% |
| Weight | 25kg | 60kg | 70kg | 79kg | 214kg| 3,239 recs | Skewed Right | 77.21% |

**Summary of Geographic Representation**

- Distinct Teams: 1184
- Distinct NOCs: 230
- Distinct Regions: 209
- Distinct Cities that hosted the Olympics: 42

**<center>Top 10 Regions by Team Associations:</center>**

| Region  | Number of Teams Associated |
|---------|----------------------------| 
| France  | 162                        |
| USA     | 97                         |
| UK      | 94                         |
| Sweden  | 52                         |
| Norway  | 46                         |
| Germany | 43                         |
| Italy   | 36                         |
| Canada  | 32                         |
| Greece  | 29                         |
| Denmark | 29                         |

**Sport Coverage:**
- Total Sports: 66
- Total Events: 765
- Average Events per Sport: ~11.6

**Sport Categories:**
- Summer Sports: 52 (including discontinued sports, 38 if excluding)
- Winter Sports: 17 (including discontinued sports, 15 if excluding)
- Discontinued Sports: 15 (e.g., Art Competitions, Tug-Of-War)

**<center>Most Event-Dense Sports:</center>**
    
|Sport|Number of Distinct Events in Sport|
|-|-|
|Shooting|83|
|Athletics|83|
|Swimming|55|
|Cycling|44|
|Sailing|38|

**<center>Medal Distribution</center>**

| Medal | Records |
| ----- | ------- |
| Bronze | 13,287 |
| Silver | 13,100 |
| Gold | 13,368
| No Medal | 229,906 |

## V. Entity Relationship Diagram
    
![SportsStats ERD](SportsStats_ERD.png)

The image you see here is the Entity Relationship Diagram that represents the connection between ath_events and noc_regions in the olympics.db database. The connection between NOC from ath_events and NOC from noc_regions is the only connection between the two tables, and they share a many-to-one relationship respectively.